In [31]:
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=FitFailedWarning)

In [2]:
info = pd.read_excel("Final Cleaned Data.xlsx")
info.head()

,Unnamed: 0,ASIN,Description,Price,Verified Subcategory,Description_New,Description_Count,Description_Ngram
0,0,B0192Y9XSE,Connects a DisplayPort DP equipped laptop or d...,13.99,DP and MDP Display Adapters,connect a displayport dp equipped laptop or de...,"(0, 1283)\t4\n (0, 1519)\t2\n (0, 1564)\t1...","(0, 4254)\t0.07386659530351065\n (0, 941)\t..."
1,1,B017Q8ZVWK,COMPACT DESIGN - The compact-designed portable...,9.99,DP and MDP Display Adapters,compact design-the compact-design portable ben...,"(0, 1283)\t4\n (0, 1519)\t2\n (0, 1564)\t1...","(0, 4254)\t0.07386659530351065\n (0, 941)\t..."
2,2,B014RWWURC,DisplayPort to DVI cable transmits HD video fr...,12.00,DP and MDP Display Adapters,displayport to dvi cable transmit hd video fro...,"(0, 1283)\t4\n (0, 1519)\t2\n (0, 1564)\t1...","(0, 4254)\t0.07386659530351065\n (0, 941)\t..."
3,3,B07YDNSGY2,USB C to Displayport Ultra High Definition:Res...,17.99,USB-C Display Adapters,usb c to displayport ultra high definition res...,"(0, 1283)\t4\n (0, 1519)\t2\n (0, 1564)\t1...","(0, 4254)\t0.07386659530351065\n (0, 941)\t..."
4,4,B00Z08TZNU,"Connect a DisplayPort (DP, DP++, DisplayPort++...",8.99,DP and MDP Display Adapters,connect a displayport dp dp++ displayport++ eq...,"(0, 1283)\t4\n (0, 1519)\t2\n (0, 1564)\t1...","(0, 4254)\t0.07386659530351065\n (0, 941)\t..."


In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
X = info["Price"].values.reshape(-1,1)
y = info['Verified Subcategory']

knn_param = {
    'n_neighbors': [5,15,25,35,50, 75, 100, 120, 140, 160, 180, 200],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1,2]
}
knn = KNeighborsClassifier(n_jobs=-1)

lr_param = {
    'C': [0, 10, 100, 1000, 1300, 1500, 1700, 2000],
    'max_iter': [100, 200, 300, 400, 600, None],
    'fit_intercept' : [True, False],
    'multi_class' : ['auto', 'ovr', 'multinomial'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

lr = LogisticRegression()

rd_knn = RandomizedSearchCV(estimator=knn, param_distributions=knn_param, cv=10, n_iter=20, n_jobs=-1)
rd_lr = RandomizedSearchCV(estimator=lr, param_distributions=lr_param, cv=10, n_iter=20, n_jobs=-1)

rd_knn.fit(X, y)
rd_lr.fit(X, y)

print("KNN score", rd_knn.best_score_)
print("KNN best parameters", rd_knn.best_params_)
print("LR score", rd_lr.best_score_)
print("LR best parameters", rd_lr.best_params_)

KNN score 0.46063951004130466
KNN best parameters {'p': 2, 'n_neighbors': 200, 'algorithm': 'auto'}
LR score 0.4100056971941319
LR best parameters {'solver': 'saga', 'multi_class': 'auto', 'max_iter': 200, 'fit_intercept': True, 'C': 2000}


In [18]:
from sklearn.ensemble import VotingClassifier

vote = VotingClassifier(
    estimators = [
        ('knn', rd_knn.best_estimator_),
        ('lr', rd_lr.best_estimator_)
    ],
    voting='soft'
)

score_vt = cross_val_score(vote, X, y, n_jobs=-1, verbose=0, cv=5)
print("Voting score:", score_vt.mean())

Voting score: 0.4506329113924051


In [34]:
'''
    This score actually make sense since we only have one feature.
    We will save this model for later use.
'''

import joblib

joblib.dump(rd_knn, "Price_Knn.pkl")

['Price_Knn.pkl']